In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [2]:
import pandas as pd
df = pd.read_csv('Dataset.csv')

In [3]:
df.head()

,Commodity,State,Market,Today Price,Last Week,Last Month,Last Year,Today Arrivals,Last Week.1,Last Month.1,Last Year.1
0,Bajra(Pearl Millet/Cumbu),Andhra Pradesh,Kurnool,2379 (Mx:2657 Mn:2259),2389 (Mx:2625 Mn:2389),-,-,10.6,1.4,-,-
1,Bajra(Pearl Millet/Cumbu),Chattisgarh,Raipur,2968 (Mx:3150 Mn:2900),-,-,-,48.8,-,-,-
2,Bajra(Pearl Millet/Cumbu),Gujarat,Amirgadh,2685 (Mx:2685 Mn:2680),2650 (Mx:2650 Mn:2640),2650 (Mx:2650 Mn:2600),-,0.8,1.4,0.4,-
3,Bajra(Pearl Millet/Cumbu),Gujarat,Amreli,2705 (Mx:3125 Mn:1830),-,2840 (Mx:3000 Mn:2440),-,1.8,-,4.4,-
4,Bajra(Pearl Millet/Cumbu),Gujarat,Bayad,2100 (Mx:2625 Mn:2000),2300 (Mx:2625 Mn:2000),-,-,0.1,0.1,-,-


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3623 entries, 0 to 3622
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Commodity       3623 non-null   object
 1   State           3623 non-null   object
 2   Market          3623 non-null   object
 3   Today Price     3616 non-null   object
 4   Last Week       3615 non-null   object
 5   Last Month      3618 non-null   object
 6   Last Year       3623 non-null   object
 7   Today Arrivals  3623 non-null   object
 8   Last Week.1     3623 non-null   object
 9   Last Month.1    3616 non-null   object
 10  Last Year.1     3521 non-null   object
dtypes: object(11)
memory usage: 311.5+ KB


In [5]:
df.describe()

,Commodity,State,Market,Today Price,Last Week,Last Month,Last Year,Today Arrivals,Last Week.1,Last Month.1,Last Year.1
count,3623,3623,3623,3616,3615,3618,3623,3623,3623,3616,3521
unique,19,27,1587,2459,1985,1651,688,1090,997,893,353
top,Onion,Uttar Pradesh,Kanpur(Grain),3000 (Mx:3000 Mn:2500),-,-,-,1,-,-,-
freq,684,641,11,48,730,1384,2703,117,726,1381,2616


In [6]:
df.rename(columns={
    'Today Price': 'Price',
    'Market': 'Market',
    'State': 'State',
    'Commodity': 'Commodity'
}, inplace=True)

In [7]:
df.head()

,Commodity,State,Market,Price,Last Week,Last Month,Last Year,Today Arrivals,Last Week.1,Last Month.1,Last Year.1
0,Bajra(Pearl Millet/Cumbu),Andhra Pradesh,Kurnool,2379 (Mx:2657 Mn:2259),2389 (Mx:2625 Mn:2389),-,-,10.6,1.4,-,-
1,Bajra(Pearl Millet/Cumbu),Chattisgarh,Raipur,2968 (Mx:3150 Mn:2900),-,-,-,48.8,-,-,-
2,Bajra(Pearl Millet/Cumbu),Gujarat,Amirgadh,2685 (Mx:2685 Mn:2680),2650 (Mx:2650 Mn:2640),2650 (Mx:2650 Mn:2600),-,0.8,1.4,0.4,-
3,Bajra(Pearl Millet/Cumbu),Gujarat,Amreli,2705 (Mx:3125 Mn:1830),-,2840 (Mx:3000 Mn:2440),-,1.8,-,4.4,-
4,Bajra(Pearl Millet/Cumbu),Gujarat,Bayad,2100 (Mx:2625 Mn:2000),2300 (Mx:2625 Mn:2000),-,-,0.1,0.1,-,-


In [8]:
import re

def remove_brackets(val):
    # Remove anything in parentheses/brackets and trailing/leading spaces
    if pd.isnull(val) or str(val).strip() == "-":
        return None
    # Remove bracketed info and keep only the first numeric value
    val = re.sub(r"\s*\(.*?\)", "", str(val)).strip()
    try:
        return float(val)
    except:
        return None

for col in ['Price', 'Last Week', 'Last Month', 'Last Year']:
    if col in df.columns:
        df[col] = df[col].apply(remove_brackets)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3623 entries, 0 to 3622
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Commodity       3623 non-null   object 
 1   State           3623 non-null   object 
 2   Market          3623 non-null   object 
 3   Price           3616 non-null   float64
 4   Last Week       2885 non-null   float64
 5   Last Month      2234 non-null   float64
 6   Last Year       906 non-null    float64
 7   Today Arrivals  3623 non-null   object 
 8   Last Week.1     3623 non-null   object 
 9   Last Month.1    3616 non-null   object 
 10  Last Year.1     3521 non-null   object 
dtypes: float64(4), object(7)
memory usage: 311.5+ KB


In [10]:
df.head()

,Commodity,State,Market,Price,Last Week,Last Month,Last Year,Today Arrivals,Last Week.1,Last Month.1,Last Year.1
0,Bajra(Pearl Millet/Cumbu),Andhra Pradesh,Kurnool,2379.0,2389.0,NaN,NaN,10.6,1.4,-,-
1,Bajra(Pearl Millet/Cumbu),Chattisgarh,Raipur,2968.0,NaN,NaN,NaN,48.8,-,-,-
2,Bajra(Pearl Millet/Cumbu),Gujarat,Amirgadh,2685.0,2650.0,2650.0,NaN,0.8,1.4,0.4,-
3,Bajra(Pearl Millet/Cumbu),Gujarat,Amreli,2705.0,NaN,2840.0,NaN,1.8,-,4.4,-
4,Bajra(Pearl Millet/Cumbu),Gujarat,Bayad,2100.0,2300.0,NaN,NaN,0.1,0.1,-,-


In [11]:
numerical_cols = [
    'Price', 'Last Week', 'Last Month', 'Last Year'
]

for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Now, fill NaN values in each numerical column with that column's median
for col in numerical_cols:
    median_value = df[col].median()
    df[col].fillna(median_value, inplace=True)

print(df.isnull().sum())

Commodity           0
State               0
Market              0
Price               0
Last Week           0
Last Month          0
Last Year           0
Today Arrivals      0
Last Week.1         0
Last Month.1        7
Last Year.1       102
dtype: int64


C:\Users\priya\AppData\Local\Temp\ipykernel_18604\629278663.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_value, inplace=True)
C:\Users\priya\AppData\Local\Temp\ipykernel_18604\629278663.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [12]:
df.head()

,Commodity,State,Market,Price,Last Week,Last Month,Last Year,Today Arrivals,Last Week.1,Last Month.1,Last Year.1
0,Bajra(Pearl Millet/Cumbu),Andhra Pradesh,Kurnool,2379.0,2389.0,2500.0,2410.0,10.6,1.4,-,-
1,Bajra(Pearl Millet/Cumbu),Chattisgarh,Raipur,2968.0,2520.0,2500.0,2410.0,48.8,-,-,-
2,Bajra(Pearl Millet/Cumbu),Gujarat,Amirgadh,2685.0,2650.0,2650.0,2410.0,0.8,1.4,0.4,-
3,Bajra(Pearl Millet/Cumbu),Gujarat,Amreli,2705.0,2520.0,2840.0,2410.0,1.8,-,4.4,-
4,Bajra(Pearl Millet/Cumbu),Gujarat,Bayad,2100.0,2300.0,2500.0,2410.0,0.1,0.1,-,-


In [13]:
import pandas as pd

# Assume 'df' is your DataFrame after handling missing values
# Define your categorical features
categorical_cols = ['Commodity', 'State', 'Market']

# Perform one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Display the first few rows to see the transformation
print("Data after One-Hot Encoding:")
print(df_encoded.head())

# # Display the new column names
print("\nNew columns created by encoding:")
print(df_encoded.columns)

Data after One-Hot Encoding:
    Price  Last Week  Last Month  Last Year Today Arrivals Last Week.1  \
0  2379.0     2389.0      2500.0     2410.0           10.6         1.4   
1  2968.0     2520.0      2500.0     2410.0           48.8           -   
2  2685.0     2650.0      2650.0     2410.0            0.8         1.4   
3  2705.0     2520.0      2840.0     2410.0            1.8           -   
4  2100.0     2300.0      2500.0     2410.0            0.1         0.1   

  Last Month.1 Last Year.1  Commodity_Bajra(Pearl Millet/Cumbu)  \
0            -           -                                 True   
1            -           -                                 True   
2          0.4           -                                 True   
3          4.4           -                                 True   
4            -           -                                 True   

   Commodity_Barley (Jau)  ...  Market_Yadgir  Market_Yalburga  \
0                   False  ...          False            

In [14]:
df_encoded.drop(columns=['Today Arrivals', 'Last Week.1', 'Last Month.1', 'Last Year.1'], inplace=True)

In [15]:
df_encoded.head()

,Price,Last Week,Last Month,Last Year,Commodity_Bajra(Pearl Millet/Cumbu),Commodity_Barley (Jau),Commodity_Bengal Gram(Gram)(Whole),Commodity_Black Gram (Urd Beans)(Whole),Commodity_Copra,Commodity_Cotton,...,Market_Yadgir,Market_Yalburga,Market_Yamuna Nagar,Market_Yawal,Market_Yeola,Market_Yeotmal,Market_Zaheerabad,Market_Zalod(Sanjeli),Market_Zalod(Zalod),Market_Zira
0,2379.0,2389.0,2500.0,2410.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2968.0,2520.0,2500.0,2410.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2685.0,2650.0,2650.0,2410.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2705.0,2520.0,2840.0,2410.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2100.0,2300.0,2500.0,2410.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


X = df_encoded.drop(columns=['Price'])  # All columns except the one you want to predict
y = df_encoded['Price'] 
# -->> ADD THIS NEW CODE <<--
# Calculate the mean of the numerical columns
mean_values = X[['Last Week', 'Last Month', 'Last Year']].mean()

print("--- COPY THESE VALUES ---")
print(mean_values)
print("-------------------------")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror', # Specifies the learning task and objective
    n_estimators=100,             # Number of boosting rounds
    max_depth=4,                  # Maximum depth of a tree
    learning_rate=0.1,            # Step size shrinkage
    random_state=10
)

--- COPY THESE VALUES ---
Last Week     3195.014057
Last Month    2855.132829
Last Year     2600.097847
dtype: float64
-------------------------


In [17]:
print("\nTraining the XGBoost model")
xgb_model.fit(X_train, y_train)
print("Model training complete.")


Training the XGBoost model
Model training complete.


In [18]:
y_pred = xgb_model.predict(X_test)

In [24]:
print(y_pred)

[ 2426.1938   3100.1865   6397.881    2086.408    2840.6797   6727.257
  2475.8464   4867.3276   1882.1954   1348.9619   2983.5505   2416.383
  2163.4695   2735.9104   2947.8115   4146.6094   2253.738    3229.3635
  4778.1553   2507.8347   2475.8464   2563.454    2321.9429   2806.6023
  2460.6929   3314.488    7421.2964   2359.662    7755.916    8832.101
  4250.3926   3144.77     3013.666    2206.2932   2032.5316   2677.0784
  2363.2832   2381.8115   2005.9442   2363.2832   2252.8186   1821.3854
  7360.9995   6235.015    2048.768   12490.737    7199.7246   5363.957
  1935.6183   6304.2275   2191.5881   2614.365    5994.801    5974.7153
  2789.239    2005.9442   2141.131    7525.944    2972.5757   6981.8223
  2460.6929   6720.777    2377.3718   3655.8718   2511.1128   4154.6733
  2487.7815   5379.6426   3429.9817   5762.873    6221.817    5185.0366
  8106.2905   2614.916    1583.7046   2537.7002   2244.8157   2735.9104
  6272.76     2891.5522   2041.5914   3172.805    2735.9104   3397.5

In [19]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # Or mse**(0.5)
mae = mean_absolute_error(y_test, y_pred)

print("\nModel Performance Evaluation:")
print(f"  Mean Squared Error (MSE): {mse:.2f}")
print(f"  Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"  Mean Absolute Error (MAE): {mae:.2f}")


Model Performance Evaluation:
  Mean Squared Error (MSE): 706309.29
  Root Mean Squared Error (RMSE): 840.42
  Mean Absolute Error (MAE): 433.93


In [ ]:
query_df = pd.DataFrame(columns=xgb_model.columns)
query_df.loc[0, :] = 0.0
query_df.loc[0, commodity_col] = 1.0

In [21]:
import joblib
import xgboost as xgb
import numpy as np

# Assume 'final_xgb_model' is your trained XGBoost model from the previous step
# Assume 'X_test' is your test data for making a sample prediction

# --- 1. Save the model as a .pkl file (Recommended) ---
pkl_filename = "xgb_price_predictor.pkl"
joblib.dump(xgb_model, pkl_filename)

print(f"Model saved successfully as: {pkl_filename}")


# --- 2. Save the model in XGBoost's native format (Alternative) ---
json_filename = "xgb_price_predictor.json"
xgb_model.save_model(json_filename)

print(f"Model saved successfully in native XGBoost format as: {json_filename}")


# --- How to Load and Use Your Saved Models ---

# Load the .pkl model
loaded_model_pkl = joblib.load(pkl_filename)
print("\nLoaded .pkl model successfully.")

# Load the native XGBoost model
loaded_model_native = xgb.XGBRegressor()
loaded_model_native.load_model(json_filename)
print("Loaded native XGBoost model successfully.")


# --- Make a test prediction to verify ---
# (Using the first row of your test data as an example)
sample_prediction_pkl = loaded_model_pkl.predict(X_test[:1])
sample_prediction_native = loaded_model_native.predict(X_test[:1])

print(f"\nTest prediction from .pkl model: {sample_prediction_pkl[0]:.2f}")
print(f"Test prediction from native model: {sample_prediction_native[0]:.2f}")



Model saved successfully as: xgb_price_predictor.pkl
Model saved successfully in native XGBoost format as: xgb_price_predictor.json

Loaded .pkl model successfully.
Loaded native XGBoost model successfully.

Test prediction from .pkl model: 2426.19
Test prediction from native model: 2426.19


In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import joblib

# Assume 'X' and 'y' are your fully preprocessed features and target
# 1. Split the data (if you haven't already)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

# 2. Initialize and train the Random Forest Regressor
# These are solid starting parameters
rf_model = RandomForestRegressor(
    n_estimators=100,      # The number of trees in the forest
    max_depth=5,          # The maximum depth of each tree
    random_state=12,
    n_jobs=-1              # Use all available CPU cores
)

print("Training the Random Forest model...")
rf_model.fit(X_train, y_train)
print("Model training complete.")

# 3. Evaluate the model's performance
y_pred_rf = rf_model.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

print("\nRandom Forest Performance:")
print(f"  Root Mean Squared Error (RMSE): {rmse_rf:.2f}")

# 4. Save the model to a .pkl file
pkl_filename_rf = "random_forest_predictor.pkl"
joblib.dump(rf_model, pkl_filename_rf)

print(f"\nModel saved successfully as: {pkl_filename_rf}")


Training the Random Forest model...
Model training complete.

Random Forest Performance:
  Root Mean Squared Error (RMSE): 983.07

Model saved successfully as: random_forest_predictor.pkl


In [25]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib  # <-- Library for saving the model
import os

# --- 1. DATA PREPARATION ---
# Assume 'df' is your initial DataFrame loaded from a CSV
# e.g., df = pd.read_csv('your_data.csv')

# Make sure the columns exist before dropping.
# This step is from your screenshot, cleaning up duplicate/unwanted columns.
cols_to_drop = ['Today Arrivals', 'Last Week.1', 'Last Month.1', 'Last Year.1']
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
if existing_cols_to_drop:
    df.drop(columns=existing_cols_to_drop, inplace=True)
    print(f"Dropped columns: {existing_cols_to_drop}")

# Perform one-hot encoding, exactly as you did
categorical_cols = ['Commodity', 'State', 'Market']
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Convert boolean columns created by get_dummies to integers (0s and 1s)
# This is a robust practice to ensure all data is numeric.
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'bool':
        df_encoded[col] = df_encoded[col].astype(int)

print("\nData after One-Hot Encoding:")
print(df_encoded.head())


# --- 2. PREPARE DATA FOR MODELING ---

# Define your features (X) and target (y)
X = df_encoded.drop(columns=['Price'])
y = df_encoded['Price']

# --- !!! THIS IS THE CRITICAL STEP FOR YOUR FLASK APP !!! ---
# Calculate the mean of the numerical columns needed for defaults in Flask.
mean_values = X[['Last Week', 'Last Month', 'Last Year']].mean()

print("\n" + "="*50)
print("--- COPY THESE VALUES INTO YOUR FLASK APP'S DEFAULT_VALUES ---")
print(mean_values)
print("="*50 + "\n")
# ---

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)


# --- 3. MODEL TRAINING ---

# Initialize the XGBoost Regressor with your parameters
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=10
)

print("Training the XGBoost model...")
# Fit the model on the training data
xgb_model.fit(X_train, y_train)
print("Model training complete.")


# --- 4. MODEL EVALUATION (Good practice to check performance) ---

y_pred = xgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("\nModel Performance on Test Set:")
print(f"  Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"  Mean Absolute Error (MAE): {mae:.2f}")


# --- 5. SAVE THE TRAINED MODEL TO A FILE ---

# Define the filename for your model
model_filename = 'xgb_price_predictor_g.pkl'

# Use joblib to dump the trained model into a file
joblib.dump(xgb_model, model_filename)

print("\n" + "*"*50)
print(f"✅ Model successfully saved to '{model_filename}'")
print("➡️ MOVE THIS FILE to your 'GRAMINAI/backend/' directory.")
print("*"*50)


Dropped columns: ['Today Arrivals', 'Last Week.1', 'Last Month.1', 'Last Year.1']

Data after One-Hot Encoding:
    Price  Last Week  Last Month  Last Year  \
0  2379.0     2389.0      2500.0     2410.0   
1  2968.0     2520.0      2500.0     2410.0   
2  2685.0     2650.0      2650.0     2410.0   
3  2705.0     2520.0      2840.0     2410.0   
4  2100.0     2300.0      2500.0     2410.0   

   Commodity_Bajra(Pearl Millet/Cumbu)  Commodity_Barley (Jau)  \
0                                    1                       0   
1                                    1                       0   
2                                    1                       0   
3                                    1                       0   
4                                    1                       0   

   Commodity_Bengal Gram(Gram)(Whole)  \
0                                   0   
1                                   0   
2                                   0   
3                                   0   
4  